In [1]:
import argparse
import numpy as np

import torch

from utils import load_data,svdApprox
from GNN import GNN_embed
import pycombo
import networkx as nx

In [2]:
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

In [3]:
# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=426, help='Random seed.')
parser.add_argument('--epochs', type=int, default=20001,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.00001,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=10e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--ndim', type=int, default=2,
                    help='Embeddings dimension.')

args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [4]:
if 1:
    # Taxi Dataset Graph
    adj = load_data()
    #adj = load_data(daily=True)
else:
    #Toy Example Graph
    adj = toy_data()

In [5]:
#partition using pycombo
G = nx.from_numpy_matrix(np.array(adj[0]))
partition = pycombo.execute(G)

In [6]:
#get binary matrix of the partition
nb_community = max(list(partition[0].values())) + 1
communities =  np.array(list(partition[0].values())).reshape(-1)
C = np.eye(nb_community)[communities]

In [7]:
#build the aggregated network
adj_agg = np.matmul(np.matmul(np.transpose(C),adj[0]),C)

In [8]:
adj_agg = torch.FloatTensor(adj_agg)
adj_agg = adj_agg.unsqueeze(0)

In [9]:
#run gnn on aggr network
agg_embed = GNN_embed(adj=adj_agg,dim=args.ndim)

Fixed Sigma dim 2
Epoch: 0001 loss: 16.61537933 time: 0.0000s
Epoch: 5001 loss: 16.61537933 time: 0.0000s
Epoch: 10001 loss: 16.61537933 time: 0.0000s
Epoch: 15001 loss: 16.61537933 time: 0.0000s
Epoch: 20001 loss: 16.61537933 time: 0.0000s
Optimization Finished!
Total time elapsed: 11.7072s


In [10]:
agg_embed = agg_embed[0].detach().numpy()

In [11]:
#perform svd on orig network
_,sig,_,svd_embedx,svd_embedy = svdApprox(adj,2)
svd_embed = torch.cat([svd_embedx,svd_embedy],dim=1)
svd_embed = svd_embed.detach().numpy()
#sigma = sig * torch.ones(svd_embed.shape)
#svd_embed = torch.cat([svd_embed,sigma],dim=1)
#svd_embed = svd_embed.numpy()

In [12]:
#addition features to be added for disagg network
disagg_embed = agg_embed[communities]*np.absolute(svd_embed)/sum(np.absolute(svd_embed).sum(axis=1))

In [13]:
#combine svd embed with additional features above
svd_embedx,svd_embedy = np.split(svd_embed,2,axis=1)
disagg_embedx,disagg_embedy = np.split(disagg_embed,2,axis=1)

In [14]:
featuresx = np.concatenate([svd_embedx,disagg_embedx],axis=1)
featuresy = np.concatenate([svd_embedy,disagg_embedy],axis=1)

In [15]:
features = np.concatenate([featuresx,featuresy],axis=1)

In [17]:
features = torch.FloatTensor(features)
features = features.unsqueeze(0)

In [18]:
#run gnn on disagg network
embed = GNN_embed(adj=adj,dim=args.ndim,features=features)

Flexible Sigma dim 2
Epoch: 0001 loss: 219041218166784.00000000 time: 0.0000s
Epoch: 5001 loss: 219029138571264.00000000 time: 0.0000s
Epoch: 10001 loss: 219029138571264.00000000 time: 0.0015s


KeyboardInterrupt: 

In [23]:
features[0][0]

tensor([   -20.4785,     -0.2805,     -5.2281,     -0.0180,    -20.4779,
            -0.2806,     -5.2279,     -0.0180])